In [2]:
import duckdb

In [3]:
duckdb.read_parquet('playlists.parquet')

┌──────────────────────┬───────────────┬────────┬─────────────┬───┬─────────────┬─────────────┬─────────────┐
│         name         │ collaborative │  pid   │ modified_at │ … │ duration_ms │ num_artists │ description │
│       varchar        │    varchar    │ int64  │    int64    │   │    int64    │    int64    │   varchar   │
├──────────────────────┼───────────────┼────────┼─────────────┼───┼─────────────┼─────────────┼─────────────┤
│ Throwbacks           │ false         │      0 │  1493424000 │ … │    11532414 │          37 │ NULL        │
│ Awesome Playlist     │ false         │      1 │  1506556800 │ … │    11656470 │          21 │ NULL        │
│ korean               │ false         │      2 │  1505692800 │ … │    14039958 │          31 │ NULL        │
│ mat                  │ false         │      3 │  1501027200 │ … │    28926058 │          86 │ NULL        │
│ 90s                  │ false         │      4 │  1401667200 │ … │     4335282 │          16 │ NULL        │
│ Wedding 

In [3]:
# informations sur le type de données
duckdb.sql("DESCRIBE SELECT * FROM './playlists.parquet'")


┌───────────────┬──────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │                         column_type                          │  null   │   key   │ default │  extra  │
│    varchar    │                           varchar                            │ varchar │ varchar │ varchar │ varchar │
├───────────────┼──────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ name          │ VARCHAR                                                      │ YES     │ NULL    │ NULL    │ NULL    │
│ collaborative │ VARCHAR                                                      │ YES     │ NULL    │ NULL    │ NULL    │
│ pid           │ BIGINT                                                       │ YES     │ NULL    │ NULL    │ NULL    │
│ modified_at   │ BIGINT                                                       │ YES     │ NULL    │ NULL    │ NULL    │
│ num_tracks    │ BIGINT        

In [6]:
# Statistiques de notre base de données (10 %)
stats_df = duckdb.sql("SUMMARIZE SELECT * FROM './playlists.parquet' USING SAMPLE 10%;").df()

stats_df.head(20)

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,name,VARCHAR,(Nuevo),🦋🦋🦋,30354,None,None,None,None,None,103808,0.00
1,collaborative,VARCHAR,false,true,2,None,None,None,None,None,103808,0.00
2,pid,BIGINT,6000,999999,103780,538916.6313193588,300504.5814002386,276456,556673,800268,103808,0.00
3,modified_at,BIGINT,1293753600,1509494400,1918,1476261812.8853269,36713270.55404717,1458842813,1490264294,1505485774,103808,0.00
4,num_tracks,BIGINT,5,250,246,66.43807799013564,53.75389859894742,26,49,92,103808,0.00
5,num_albums,BIGINT,2,244,236,49.62731196054254,39.95410579232352,20,37,68,103808,0.00
6,num_followers,BIGINT,1,6611,151,2.076843788532676,34.39343790431074,1,1,1,103808,0.00
7,tracks,"STRUCT(pos BIGINT, artist_name VARCHAR, track_...","[{'pos': 0, 'artist_name': DRAM, 'track_uri': ...","[{'pos': 0, 'artist_name': DRAM, 'track_uri': ...",106047,None,None,None,None,None,103808,0.00
8,num_edits,BIGINT,2,201,183,17.623102265721332,20.666926242215997,5,10,22,103808,0.00
9,duration_ms,BIGINT,689632,216094210,103292,15595764.219077528,12853264.471245624,6019053,11438448,21391066,103808,0.00


In [9]:
# Nombre d'apparition de l'artise taylor Swift
duckdb.execute("SET memory_limit = '20GB'")

result = duckdb.sql("WITH tracks AS (SELECT UNNEST(tracks, recursive := true) FROM 'playlists.parquet') SELECT count(*) FROM tracks WHERE artist_name ilike 'Taylor Swift' ")

result.df()

,count_star()
0,105366


In [7]:
result = duckdb.sql("WITH tracks AS (SELECT UNNEST(tracks, recursive := true) FROM 'playlists.parquet') SELECT DISTINCT artist_name FROM tracks")

print("Nombre d'artistes différents au sein de la base de données : ", len(result))

Nombre d'artistes différents au sein de la base de données :  287742


In [12]:
result = duckdb.sql("WITH tracks AS (SELECT UNNEST(tracks, recursive := true) FROM 'playlists.parquet' SELECT DISTINCT track_name FROM tracks")

print("Nombre de tracks différents au sein de la base de données : ", len(result))

Nombre de tracks différents au sein de la base de données :  1483760


In [5]:
duckdb.execute("SET memory_limit = '20GB'")

tracks_df = duckdb.sql("WITH tracks AS (SELECT UNNEST(tracks, recursive := true) FROM 'playlists.parquet') SELECT * FROM tracks").df()

tracks_df.head()

: 